In [1]:
from scipy.stats import ks_2samp
from numpy.random import seed, randn, lognormal



In [2]:
# from scipy.stats import wilcoxon as wl 
import pandas as pd 
from scipy.stats import ks_2samp

In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np 

df = pd.read_csv('./data/df.csv',sep=',')
df  = df.sort_values(by='time',ascending=True).reset_index(drop=True)

BATCHSIZE = 256
LOOKBACK = 10


date_target = ["2024-02-19",
"2024-03-15",
"2024-04-19",
"2024-05-17",
"2024-06-21",
"2024-07-19",
"2024-08-16",
"2024-09-20",
"2024-10-18",
"2024-11-15",
"2024-12-20",
"2025-01-17"]


df['time'] = pd.to_datetime(df['time'])
df['date'] = df['time'].dt.date
print(df.head())
df['date'] = df['date'].astype(str)
df['hour'] = df['time'].dt.hour
df['minute'] = df['time'].dt.minute 
# for i in range(1,5):
#     df[f'spread_shift_{i}'] = df.groupby(['hour','minute'])['spread'].shift(i)
df['std'] = df['spread'].rolling(window=200).std()
df['volatility'] = df['std'].apply(lambda x: np.log(x))

df['Expiration_Date'] = df['date'].apply(lambda x: 1 if x in date_target else 0)
# df.fillna(0,inplace=True)


df.dropna(inplace=True)

df.head()


                 time  IF00_close  IF01_close  spread        date
0 2024-01-24 09:30:00      3240.0      3236.8    -3.2  2024-01-24
1 2024-01-24 09:31:00      3241.4      3243.0     1.6  2024-01-24
2 2024-01-24 09:32:00      3240.0      3243.0     3.0  2024-01-24
3 2024-01-24 09:33:00      3234.4      3235.6     1.2  2024-01-24
4 2024-01-24 09:34:00      3235.8      3236.0     0.2  2024-01-24


,time,IF00_close,IF01_close,spread,date,hour,minute,std,volatility,Expiration_Date
199,2024-01-24 14:19:00,3250.6,3245.8,-4.8,2024-01-24,14,19,2.284265,0.826044,0
200,2024-01-24 14:20:00,3245.8,3241.0,-4.8,2024-01-24,14,20,2.294348,0.830449,0
201,2024-01-24 14:21:00,3243.2,3239.0,-4.2,2024-01-24,14,21,2.296284,0.831292,0
202,2024-01-24 14:22:00,3240.8,3235.0,-5.8,2024-01-24,14,22,2.300440,0.833100,0
203,2024-01-24 14:23:00,3243.8,3238.0,-5.8,2024-01-24,14,23,2.316842,0.840205,0


In [4]:
df.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58364 entries, 199 to 58562
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   time             58364 non-null  datetime64[ns]
 1   IF00_close       58364 non-null  float64       
 2   IF01_close       58364 non-null  float64       
 3   spread           58364 non-null  float64       
 4   date             58364 non-null  object        
 5   hour             58364 non-null  int64         
 6   minute           58364 non-null  int64         
 7   std              58364 non-null  float64       
 8   volatility       58364 non-null  float64       
 9   Expiration_Date  58364 non-null  int64         
dtypes: datetime64[ns](1), float64(5), int64(3), object(1)
memory usage: 4.9+ MB


In [5]:
df['year'] = df['time'].dt.year 
df['month'] = df['time'].dt.month 

In [6]:
df['year_month'] = df[['year','month']].apply(lambda x: str(x[0])+ '0'+ str(x[1]) if x[1]<=9 else str(x[0])+str(x[1]),axis=1)
df 

,time,IF00_close,IF01_close,spread,date,hour,minute,std,volatility,Expiration_Date,year,month,year_month
199,2024-01-24 14:19:00,3250.6,3245.8,-4.8,2024-01-24,14,19,2.284265,0.826044,0,2024,1,202401
200,2024-01-24 14:20:00,3245.8,3241.0,-4.8,2024-01-24,14,20,2.294348,0.830449,0,2024,1,202401
201,2024-01-24 14:21:00,3243.2,3239.0,-4.2,2024-01-24,14,21,2.296284,0.831292,0,2024,1,202401
202,2024-01-24 14:22:00,3240.8,3235.0,-5.8,2024-01-24,14,22,2.300440,0.833100,0,2024,1,202401
203,2024-01-24 14:23:00,3243.8,3238.0,-5.8,2024-01-24,14,23,2.316842,0.840205,0,2024,1,202401
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58558,2025-01-24 14:56:00,3835.2,3837.8,2.6,2025-01-24,14,56,0.527048,-0.640463,0,2025,1,202501
58559,2025-01-24 14:57:00,3834.4,3837.8,3.4,2025-01-24,14,57,0.528291,-0.638108,0,2025,1,202501
58560,2025-01-24 14:58:00,3835.0,3838.0,3.0,2025-01-24,14,58,0.528291,-0.638108,0,2025,1,202501
58561,2025-01-24 14:59:00,3836.0,3839.2,3.2,2025-01-24,14,59,0.527048,-0.640463,0,2025,1,202501


In [7]:
# df_month = df[df['year_month']=='202404']

In [8]:
# df_month

In [9]:
# df_month_target = pd.DataFrame()
# df_month_test = pd.DataFrame() 
# for date in df_month['date'].unique():
#     if date in date_target:
#         df_month_target = df_month[df_month['date']==date]
#     else:
#         df_month_test = pd.concat([df_month_test,df_month[df_month['date']==date]],axis=0)

In [10]:
# df_month_target

aa = df[df['date'].isin(date_target)] 

In [11]:
for delay in range(-3,6):

    bb = pd.DataFrame()

    for date in date_target:
        if delay>0:
            date_list = sorted(df[df['date']<date]['date'].unique())
            date_ = date_list[-delay]
        elif delay<0:
            date_list = sorted(df[df['date']>date]['date'].unique())
            date_ = date_list[-delay]
        else:
            date_ = date 

        # print(date_,date )


        df_bb = df[df['date']==date_]
        bb = pd.concat([bb,df_bb],axis=0)
    bb 
    statistics = [] 
    pvalue = []

    for month in aa['year_month'].unique():
        if aa[aa['year_month']==month].shape==bb[bb['year_month']==month].shape:
            # print(month,wl(aa[aa['year_month']==month]['spread'],bb[bb['year_month']==month]['spread']))
            statistics.append(ks_2samp(aa[aa['year_month']==month]['spread'],bb[bb['year_month']==month]['spread']).statistic)
            pvalue.append(ks_2samp(aa[aa['year_month']==month]['spread'],bb[bb['year_month']==month]['spread']).pvalue)

    array_statistics = np.array(statistics)
    # array_statistics[array_statistics<14000].mean()
    array_pvalue = np.array(pvalue)

    array_pvalue[array_pvalue<0.8].mean() 
    print("T"+ str(-delay), array_statistics.mean(),array_pvalue.mean() )







T3 0.8326417704011063 3.2286517431497974e-06
T2 0.8364453665283539 7.567674208838511e-05
T1 0.8167358229598894 1.993087108229119e-15
T0 0.0 1.0
T-1 0.5034578146611342 0.0002168561747327406


/var/folders/71/8c25_pts4j92396n9h1t_88c0000gn/T/ipykernel_44122/2100894954.py:34: RuntimeWarning: Mean of empty slice.
  array_pvalue[array_pvalue<0.8].mean()
/Users/gaoxiancheng/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


T-2 0.5418395573997233 1.085568649294552e-06
T-3 0.7334024896265561 5.7030164793687365e-11
T-4 0.7244121715076072 1.2965669865468545e-07
T-5 0.7123098201936376 7.481048099883156e-06


In [12]:
for delay in [1]:

    bb = pd.DataFrame()

    for date in date_target:
        if delay>0:
            date_list = sorted(df[df['date']<date]['date'].unique())
            date_ = date_list[-delay]
        elif delay<0:
            date_list = sorted(df[df['date']>date]['date'].unique())
            date_ = date_list[-delay]
        else:
            date_ = date 


        df_bb = df[df['date']==date_]
        bb = pd.concat([bb,df_bb],axis=0)
    bb['minutes'] = bb['hour']*60 + bb['minute']
    aa['minutes'] = aa['hour']*60 + aa['minute']

    for periods in [[570, 585],[570, 600],[570, 630],[630.0, 690.0],[780, 840], [885, 900],[870, 900],[840, 900]]:
        start_min = periods[0]
        end_min = periods[1]
        statistics = [] 
        pvalue = []

        for month in aa['year_month'].unique():
            if aa[aa['year_month']==month].shape==bb[bb['year_month']==month].shape:
            # print(month,wl(aa[aa['year_month']==month]['spread'],bb[bb['year_month']==month]['spread']))
                aa_ = aa[(aa['year_month']==month) & (aa['minutes']>=start_min) & (aa['minutes']<end_min)]
                bb_ = bb[(bb['year_month']==month) & (bb['minutes']>=start_min) & (bb['minutes']<end_min)]
                # print(aa[aa['year_month']==month].head() )
            
                statistics.append(ks_2samp(aa_['spread'],bb_['spread']).statistic)
                pvalue.append(ks_2samp(aa_['spread'],bb_['spread']).pvalue)

        array_statistics = np.array(statistics)
        array_pvalue = np.array(pvalue)


        print("T"+ str(-delay),periods, array_statistics.mean(),array_pvalue.mean() )
        # print(array_statistics, array_pvalue)









T-1 [570, 585] 0.7444444444444445 0.062167598261412806
T-1 [570, 600] 0.7055555555555556 0.07861435958901222
T-1 [570, 630] 0.7152777777777778 0.009984255939304767
T-1 [630.0, 690.0] 0.7430555555555557 0.0007252878009401053
T-1 [780, 840] 0.751412429378531 0.0007456473527957754
T-1 [885, 900] 0.8166666666666668 0.10401278119110381
T-1 [870, 900] 0.813888888888889 0.02130135172184772
T-1 [840, 900] 0.7194444444444446 0.024563013535514613


/var/folders/71/8c25_pts4j92396n9h1t_88c0000gn/T/ipykernel_44122/2028095792.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aa['minutes'] = aa['hour']*60 + aa['minute']


In [13]:
for delay in [2]:

    bb = pd.DataFrame()

    for date in date_target:
        if delay>0:
            date_list = sorted(df[df['date']<date]['date'].unique())
            date_ = date_list[-delay]
        elif delay<0:
            date_list = sorted(df[df['date']>date]['date'].unique())
            date_ = date_list[-delay]
        else:
            date_ = date 


        df_bb = df[df['date']==date_]
        bb = pd.concat([bb,df_bb],axis=0)
    bb['minutes'] = bb['hour']*60 + bb['minute']
    aa['minutes'] = aa['hour']*60 + aa['minute']

    for periods in [[570, 585],[570, 600],[570, 630],[630.0, 690.0],[780, 840], [885, 900],[870, 900],[840, 900]]:
        start_min = periods[0]
        end_min = periods[1]
        statistics = [] 
        pvalue = []

        for month in aa['year_month'].unique():
            if aa[aa['year_month']==month].shape==bb[bb['year_month']==month].shape:
            # print(month,wl(aa[aa['year_month']==month]['spread'],bb[bb['year_month']==month]['spread']))
                aa_ = aa[(aa['year_month']==month) & (aa['minutes']>=start_min) & (aa['minutes']<end_min)]
                bb_ = bb[(bb['year_month']==month) & (bb['minutes']>=start_min) & (bb['minutes']<end_min)]
                # print(aa[aa['year_month']==month].head() )
            
                statistics.append(ks_2samp(aa_['spread'],bb_['spread']).statistic)
                pvalue.append(ks_2samp(aa_['spread'],bb_['spread']).pvalue)

        array_statistics = np.array(statistics)
        array_pvalue = np.array(pvalue)


        print("T"+ str(-delay),periods, array_statistics.mean(),array_pvalue.mean() )
        # print(array_statistics, array_pvalue)









T-2 [570, 585] 0.7833333333333333 0.0781990852269084
T-2 [570, 600] 0.7833333333333335 0.08572982913661313
T-2 [570, 630] 0.7916666666666666 0.08294639355337495
T-2 [630.0, 690.0] 0.8013888888888888 0.0004084586133532387
T-2 [780, 840] 0.7612994350282486 0.0012100195112086799
T-2 [885, 900] 0.9611111111111111 3.381844509461815e-05
T-2 [870, 900] 0.8694444444444445 0.0018494218225707923
T-2 [840, 900] 0.7875000000000001 0.0023117208858179634


/var/folders/71/8c25_pts4j92396n9h1t_88c0000gn/T/ipykernel_44122/4166163876.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aa['minutes'] = aa['hour']*60 + aa['minute']


# 波动率



In [25]:



for delay in range(-3,6):

    bb = pd.DataFrame()

    for date in date_target:
        if delay>0:
            date_list = sorted(df[df['date']<date]['date'].unique())
            date_ = date_list[-delay]
        elif delay<0:
            date_list = sorted(df[df['date']>date]['date'].unique())
            date_ = date_list[-delay]
        else:
            date_ = date 


        df_bb = df[df['date']==date_]
        bb = pd.concat([bb,df_bb],axis=0)
    bb 
    statistics = [] 
    pvalue = []

    for month in aa['year_month'].unique():
        if aa[aa['year_month']==month].shape[0]==bb[bb['year_month']==month].shape[0]:
            # print(month,wl(aa[aa['year_month']==month]['spread'],bb[bb['year_month']==month]['spread']))
            statistics.append(ks_2samp(aa[aa['year_month']==month]['volatility'],bb[bb['year_month']==month]['volatility']).statistic)
            pvalue.append(ks_2samp(aa[aa['year_month']==month]['volatility'],bb[bb['year_month']==month]['volatility']).pvalue)
            # print("11",ks_2samp(aa[aa['year_month']==month]['volatility'],bb[bb['year_month']==month]['volatility']))
        else:
            print(aa[aa['year_month']==month].shape, bb[bb['year_month']==month].shape)

    array_statistics = np.array(statistics)
    array_statistics[array_statistics<14000].mean()
    array_pvalue = np.array(pvalue)

    array_pvalue[array_pvalue<0.8].mean() 
    print("T"+ str(-delay), array_statistics.mean(),array_pvalue.mean() )








T3 0.8675656984785616 1.0914294812247665e-35
T2 0.9049100968188104 3.038399676791729e-36
T1 0.8388658367911481 9.883160716740384e-27
T0 0.0 1.0
T-1 0.7071230982019364 6.881112199451501e-12


/var/folders/71/8c25_pts4j92396n9h1t_88c0000gn/T/ipykernel_44122/3543046309.py:35: RuntimeWarning: Mean of empty slice.
  array_pvalue[array_pvalue<0.8].mean()
/Users/gaoxiancheng/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


T-2 0.7261410788381744 2.107351068434077e-07
T-3 0.7164591977869986 6.777649153563002e-10
T-4 0.7686721991701244 3.7760351783111846e-14
T-5 0.7493084370677732 1.3291337475695026e-23


In [26]:

for delay in [1]:

    bb = pd.DataFrame()

    for date in date_target:
        if delay>0:
            date_list = sorted(df[df['date']<date]['date'].unique())
            date_ = date_list[-delay]
        elif delay<0:
            date_list = sorted(df[df['date']>date]['date'].unique())
            date_ = date_list[-delay]
        else:
            date_ = date 


        df_bb = df[df['date']==date_]
        bb = pd.concat([bb,df_bb],axis=0)
    bb['minutes'] = bb['hour']*60 + bb['minute']
    aa['minutes'] = aa['hour']*60 + aa['minute']

    for periods in [[570, 585],[570, 600],[570, 630],[630.0, 690.0],[780, 840], [885, 900],[870, 900],[840, 900]]:
        start_min = periods[0]
        end_min = periods[1]
        statistics = [] 
        pvalue = []

        for month in aa['year_month'].unique():
            if aa[aa['year_month']==month].shape[0]==bb[bb['year_month']==month].shape[0]:
            # print(month,wl(aa[aa['year_month']==month]['spread'],bb[bb['year_month']==month]['spread']))
                aa_ = aa[(aa['year_month']==month) & (aa['minutes']>=start_min) & (aa['minutes']<end_min)]
                bb_ = bb[(bb['year_month']==month) & (bb['minutes']>=start_min) & (bb['minutes']<end_min)]
                # print(aa[aa['year_month']==month].head() )
            
                statistics.append(ks_2samp(aa_['volatility'],bb_['volatility']).statistic)
                pvalue.append(ks_2samp(aa_['volatility'],bb_['volatility']).pvalue)

        array_statistics = np.array(statistics)
        array_pvalue = np.array(pvalue)


        print("T"+ str(-delay),periods, array_statistics.mean(),array_pvalue.mean() )
        # print(array_statistics, array_pvalue)









T-1 [570, 585] 0.8722222222222222 0.08041108122409384
T-1 [570, 600] 0.875 9.26078897975468e-06
T-1 [570, 630] 0.8444444444444442 9.672203251692289e-09
T-1 [630.0, 690.0] 0.9208333333333334 2.9282175368109693e-08
T-1 [780, 840] 0.8502824858757062 5.030107237329729e-07
T-1 [885, 900] 1.0 1.2893450075787697e-08
T-1 [870, 900] 1.0 1.691123389214474e-17
T-1 [840, 900] 1.0 2.0700738881869652e-35


/var/folders/71/8c25_pts4j92396n9h1t_88c0000gn/T/ipykernel_44122/2781922498.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aa['minutes'] = aa['hour']*60 + aa['minute']


In [27]:

for delay in [2]:

    bb = pd.DataFrame()

    for date in date_target:
        if delay>0:
            date_list = sorted(df[df['date']<date]['date'].unique())
            date_ = date_list[-delay]
        elif delay<0:
            date_list = sorted(df[df['date']>date]['date'].unique())
            date_ = date_list[-delay]
        else:
            date_ = date 


        df_bb = df[df['date']==date_]
        bb = pd.concat([bb,df_bb],axis=0)
    bb['minutes'] = bb['hour']*60 + bb['minute']
    aa['minutes'] = aa['hour']*60 + aa['minute']

    for periods in [[570, 585],[570, 600],[570, 630],[630.0, 690.0],[780, 840], [885, 900],[870, 900],[840, 900]]:
        start_min = periods[0]
        end_min = periods[1]
        statistics = [] 
        pvalue = []

        for month in aa['year_month'].unique():
            if aa[aa['year_month']==month].shape[0]==bb[bb['year_month']==month].shape[0]:
            # print(month,wl(aa[aa['year_month']==month]['spread'],bb[bb['year_month']==month]['spread']))
                aa_ = aa[(aa['year_month']==month) & (aa['minutes']>=start_min) & (aa['minutes']<end_min)]
                bb_ = bb[(bb['year_month']==month) & (bb['minutes']>=start_min) & (bb['minutes']<end_min)]
                # print(aa[aa['year_month']==month].head() )
            
                statistics.append(ks_2samp(aa_['volatility'],bb_['volatility']).statistic)
                pvalue.append(ks_2samp(aa_['volatility'],bb_['volatility']).pvalue)

        array_statistics = np.array(statistics)
        array_pvalue = np.array(pvalue)


        print("T"+ str(-delay),periods, array_statistics.mean(),array_pvalue.mean() )
        # print(array_statistics, array_pvalue)









T-2 [570, 585] 0.9277777777777777 0.0012760099568378863
T-2 [570, 600] 0.861111111111111 0.0030925331706191076
T-2 [570, 630] 0.7763888888888889 0.0003895985516026616
T-2 [630.0, 690.0] 0.9694444444444444 5.355291191884993e-17
T-2 [780, 840] 0.8728813559322033 1.7881786936909514e-10
T-2 [885, 900] 1.0 1.2893450075787697e-08
T-2 [870, 900] 1.0 1.691123389214474e-17
T-2 [840, 900] 0.9625 9.161523484460591e-10


/var/folders/71/8c25_pts4j92396n9h1t_88c0000gn/T/ipykernel_44122/1404791677.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aa['minutes'] = aa['hour']*60 + aa['minute']


In [42]:
ks_2samp(df_month_target['spread'].values, df_month_test_tmp.groupby('hour_minte')['spread'].mean().values )
# dangtian  和t-3之前的均值

KstestResult(statistic=0.970954356846473, pvalue=5.0614197675279473e-129, statistic_location=-4.622222222222111, statistic_sign=-1)

In [43]:
df_month_test_tmp2 = df_month_test[df_month_test['date'].isin(before_target[0:-3] )]
ks_2samp(df_month_test_tmp2.groupby('hour_minte')['spread'].mean().values,\
    df_month_test[df_month_test['date']==before_target[-1]]['spread'].values)

    # t-1  和t-3之前的均值

KstestResult(statistic=1.0, pvalue=4.409413217307657e-144, statistic_location=-3.977777777777697, statistic_sign=1)

In [44]:

df_month_test_tmp2 = df_month_test[df_month_test['date'].isin(before_target[0:-3] )]
ks_2samp(df_month_test_tmp2.groupby('hour_minte')['spread'].mean().values,\
    df_month_test[df_month_test['date']==before_target[-2]]['spread'].values)

    # t-2  和t-3之前的均值

KstestResult(statistic=0.991701244813278, pvalue=5.1114358956352085e-139, statistic_location=-4.622222222222111, statistic_sign=1)

In [45]:
after_target[0],after_target[1],after_target[2]

('2024-04-22', '2024-04-23', '2024-04-24')

In [46]:
df_month_test_tmp2 = df_month_test[df_month_test['date'].isin(before_target[0:-3] )]
ks_2samp(df_month_test_tmp2.groupby('hour_minte')['spread'].mean().values,\
    df_month_test[df_month_test['date']==after_target[0]]['spread'].values)
    # t+ 1  与和t-3之前的均值

KstestResult(statistic=1.0, pvalue=4.409413217307657e-144, statistic_location=-12.800000000000182, statistic_sign=-1)

In [47]:
df_month_test_tmp2 = df_month_test[df_month_test['date'].isin(before_target[0:-3] )]
ks_2samp(df_month_test_tmp2.groupby('hour_minte')['spread'].mean().values,\
    df_month_test[df_month_test['date']==after_target[1]]['spread'].values)
    # t+2 和t-3之前的均值

KstestResult(statistic=1.0, pvalue=4.409413217307657e-144, statistic_location=-13.0, statistic_sign=-1)

In [48]:
df_month_test_tmp2 = df_month_test[df_month_test['date'].isin(before_target[0:-3] )]
ks_2samp(df_month_test_tmp2.groupby('hour_minte')['spread'].mean().values,\
    df_month_test[df_month_test['date']==after_target[2]]['spread'].values)

    # t+3 和t-3之前的均值

KstestResult(statistic=1.0, pvalue=4.409413217307657e-144, statistic_location=-9.0, statistic_sign=-1)

## 统计部分时段内的对比

In [49]:
df_month_target

for i in range(1,3):
    for length in [15,30,60]:


        print(before_target[-i], f"前 {length} minutes",ks_2samp(df_month_test[df_month_test['date']==before_target[-i]]['spread'].values[0:length],\
            df_month_target['spread'].values[0:length]))



2024-04-18 前 15 minutes KstestResult(statistic=1.0, pvalue=1.2893450075787698e-08, statistic_location=0.8000000000001819, statistic_sign=1)
2024-04-18 前 30 minutes KstestResult(statistic=1.0, pvalue=1.6911233892144742e-17, statistic_location=0.8000000000001819, statistic_sign=1)
2024-04-18 前 60 minutes KstestResult(statistic=0.95, pvalue=5.8135955075842714e-30, statistic_location=0.2000000000002728, statistic_sign=1)
2024-04-17 前 15 minutes KstestResult(statistic=1.0, pvalue=1.2893450075787698e-08, statistic_location=-2.399999999999636, statistic_sign=1)
2024-04-17 前 30 minutes KstestResult(statistic=1.0, pvalue=1.6911233892144742e-17, statistic_location=-2.0, statistic_sign=1)
2024-04-17 前 60 minutes KstestResult(statistic=1.0, pvalue=2.070073888186965e-35, statistic_location=-2.0, statistic_sign=1)


In [50]:
for i in range(1,3):
    for length in [15,30,60]:


        print(before_target[-i], f"后 {length} minutes",ks_2samp(df_month_test[df_month_test['date']==before_target[-i]]['spread'].values[-length:],\
            df_month_target['spread'].values[-length:]))



2024-04-18 后 15 minutes KstestResult(statistic=0.3333333333333333, pvalue=0.3855465198257425, statistic_location=0.599999999999909, statistic_sign=1)
2024-04-18 后 30 minutes KstestResult(statistic=0.36666666666666664, pvalue=0.034580084963334506, statistic_location=-1.599999999999909, statistic_sign=-1)
2024-04-18 后 60 minutes KstestResult(statistic=0.25, pvalue=0.046660870723173625, statistic_location=-1.400000000000091, statistic_sign=-1)
2024-04-17 后 15 minutes KstestResult(statistic=0.8, pvalue=5.2347407307698066e-05, statistic_location=-0.3999999999996362, statistic_sign=1)
2024-04-17 后 30 minutes KstestResult(statistic=0.43333333333333335, pvalue=0.006548396368058784, statistic_location=-0.1999999999998181, statistic_sign=1)
2024-04-17 后 60 minutes KstestResult(statistic=0.5666666666666667, pvalue=3.1578017116651398e-09, statistic_location=-0.3999999999996362, statistic_sign=1)


In [52]:
df_month_target['minutes'] = df_month_target['hour']*60 + df_month_target['minute']
df_month_test['minutes'] = df_month_test['hour']*60 + df_month_test['minute']


time_ranges = [[13.0,14.0],[10.5,11.5]]

for i in range(1,3):
    for time_range in time_ranges:
        minutes_start = time_range[0]*60
        minutes_end = time_range[1]*60



        print(before_target[-i], f"{minutes_start}-{minutes_end}时间段内",  ks_2samp(df_month_test[(df_month_test['date']==before_target[-i])&
        (df_month_test['minutes']>=minutes_start)&(df_month_test['minutes']<minutes_end)]['spread'].values,\
            df_month_target[(df_month_target['minutes']>=minutes_start)&(df_month_target['minutes']<minutes_end)]['spread'].values))

2024-04-18 780.0-840.0时间段内 KstestResult(statistic=0.3050847457627119, pvalue=0.007920128345457526, statistic_location=-1.3999999999996362, statistic_sign=-1)
2024-04-18 630.0-690.0时间段内 KstestResult(statistic=0.5833333333333334, pvalue=8.642404684557224e-10, statistic_location=-0.4000000000000909, statistic_sign=1)
2024-04-17 780.0-840.0时间段内 KstestResult(statistic=0.7796610169491526, pvalue=5.7123106040051954e-18, statistic_location=-2.0, statistic_sign=1)
2024-04-17 630.0-690.0时间段内 KstestResult(statistic=1.0, pvalue=2.070073888186965e-35, statistic_location=-1.7999999999997272, statistic_sign=1)


/var/folders/71/8c25_pts4j92396n9h1t_88c0000gn/T/ipykernel_6613/2727344821.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_month_target['minutes'] = df_month_target['hour']*60 + df_month_target['minute']


# 对波动率进行做检验

## 读取数据并进行波动率的计算

In [53]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np 

df = pd.read_csv('./data/df.csv',sep=',')
df  = df.sort_values(by='time',ascending=True).reset_index(drop=True)

BATCHSIZE = 256
LOOKBACK = 10


date_target = ["2024-02-19",
"2024-03-15",
"2024-04-19",
"2024-05-17",
"2024-06-21",
"2024-07-19",
"2024-08-16",
"2024-09-20",
"2024-10-18",
"2024-11-15",
"2024-12-20",
"2025-01-17"]
df.head()

df['time'] = pd.to_datetime(df['time'])
df['date'] = df['time'].dt.date
df['date'] = df['date'].astype(str)
df['hour'] = df['time'].dt.hour
df['minute'] = df['time'].dt.minute 
# for i in range(1,5):
#     df[f'spread_shift_{i}'] = df.groupby(['hour','minute'])['spread'].shift(i)

df['Expiration_Date'] = df['date'].apply(lambda x: 1 if x in date else 0)
# df.fillna(0,inplace=True)

df['std'] = df['spread'].rolling(window=200).std()
df['volatility'] = df['std'].apply(lambda x: np.log(x))

df.dropna(inplace=True)

df.head()


,time,IF00_close,IF01_close,spread,date,hour,minute,Expiration_Date,std,volatility
199,2024-01-24 14:19:00,3250.6,3245.8,-4.8,2024-01-24,14,19,0,2.284265,0.826044
200,2024-01-24 14:20:00,3245.8,3241.0,-4.8,2024-01-24,14,20,0,2.294348,0.830449
201,2024-01-24 14:21:00,3243.2,3239.0,-4.2,2024-01-24,14,21,0,2.296284,0.831292
202,2024-01-24 14:22:00,3240.8,3235.0,-5.8,2024-01-24,14,22,0,2.300440,0.833100
203,2024-01-24 14:23:00,3243.8,3238.0,-5.8,2024-01-24,14,23,0,2.316842,0.840205


In [54]:
# df['volatility'].plot()

In [55]:
df['year'] = df['time'].dt.year 
df['month'] = df['time'].dt.month 

In [56]:
df['year_month'] = df[['year','month']].apply(lambda x: str(x[0])+ str('0'+ str(x[1] \
    if x[1]<=9 else str(x[1]))),axis=1)

/var/folders/71/8c25_pts4j92396n9h1t_88c0000gn/T/ipykernel_6613/3604794410.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df['year_month'] = df[['year','month']].apply(lambda x: str(x[0])+ str('0'+ str(x[1] \
/var/folders/71/8c25_pts4j92396n9h1t_88c0000gn/T/ipykernel_6613/3604794410.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if x[1]<=9 else str(x[1]))),axis=1)


In [57]:
df_month = df[df['year_month']=='202404']

In [58]:
df_month_target = pd.DataFrame()
df_month_test = pd.DataFrame() 
for date in df_month['date'].unique():
    if date in date_target:
        df_month_target = df_month[df_month['date']==date]
    else:
        df_month_test = pd.concat([df_month_test,df_month[df_month['date']==date]],axis=0)

In [59]:
for date in df_month_test['date'].unique():
    test_volatility = df_month_test[df_month_test['date']==date]['volatility'].values 
    target_volatility = df_month_target['volatility'].values 
    if test_volatility.shape==target_volatility.shape:
        print(date, ks_2samp(target_volatility,test_volatility))
    else:
        print(date)

2024-04-01 KstestResult(statistic=0.9875518672199171, pvalue=8.178297433016334e-137, statistic_location=-0.22284919702040262, statistic_sign=-1)
2024-04-02 KstestResult(statistic=0.970954356846473, pvalue=5.0614197675279473e-129, statistic_location=-0.13225045867194601, statistic_sign=-1)
2024-04-03 KstestResult(statistic=0.991701244813278, pvalue=5.1114358956352085e-139, statistic_location=-0.2694790116835386, statistic_sign=-1)
2024-04-08 KstestResult(statistic=0.970954356846473, pvalue=5.0614197675279473e-129, statistic_location=-0.15512314919882159, statistic_sign=-1)
2024-04-09 KstestResult(statistic=0.991701244813278, pvalue=5.1114358956352085e-139, statistic_location=-0.269223236278071, statistic_sign=-1)
2024-04-10 KstestResult(statistic=0.991701244813278, pvalue=5.1114358956352085e-139, statistic_location=-0.25199071355625297, statistic_sign=-1)
2024-04-11 KstestResult(statistic=1.0, pvalue=4.409413217307657e-144, statistic_location=-0.33035702157960833, statistic_sign=-1)
202

In [60]:
df_month_test['hour_minte'] = df_month_test['time'].apply(lambda x: str(x)[10:])
df_month_test

,time,IF00_close,IF01_close,spread,date,hour,minute,Expiration_Date,std,volatility,year,month,year_month,hour_minte
10122,2024-04-01 09:30:00,3555.6,3546.4,-9.2,2024-04-01,9,30,0,0.705752,-0.348491,2024,4,202404,09:30:00
10123,2024-04-01 09:31:00,3566.4,3559.8,-6.6,2024-04-01,9,31,0,0.705183,-0.349299,2024,4,202404,09:31:00
10124,2024-04-01 09:32:00,3573.8,3567.2,-6.6,2024-04-01,9,32,0,0.705459,-0.348907,2024,4,202404,09:32:00
10125,2024-04-01 09:33:00,3576.0,3568.2,-7.8,2024-04-01,9,33,0,0.712420,-0.339088,2024,4,202404,09:33:00
10126,2024-04-01 09:34:00,3576.0,3569.4,-6.6,2024-04-01,9,34,0,0.697141,-0.360768,2024,4,202404,09:34:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14937,2024-04-30 14:56:00,3612.0,3601.4,-10.6,2024-04-30,14,56,1,0.668480,-0.402749,2024,4,202404,14:56:00
14938,2024-04-30 14:57:00,3610.6,3600.0,-10.6,2024-04-30,14,57,1,0.669189,-0.401689,2024,4,202404,14:57:00
14939,2024-04-30 14:58:00,3608.4,3598.6,-9.8,2024-04-30,14,58,1,0.664233,-0.409123,2024,4,202404,14:58:00
14940,2024-04-30 14:59:00,3610.8,3599.8,-11.0,2024-04-30,14,59,1,0.665013,-0.407949,2024,4,202404,14:59:00


In [63]:

ks_2samp(df_month_target['volatility'].values, df_month_test.groupby('hour_minte')['volatility'].mean().values )

KstestResult(statistic=0.966804979253112, pvalue=3.005217986969719e-127, statistic_location=-0.12121833134073612, statistic_sign=-1)

In [64]:
before_target = []
after_target = []
for date in list(df_month_test['date'].unique() ):
    if date<'2024-04-19':
        before_target.append(date)
    else:
        after_target.append(date)

In [65]:
df_month_test_tmp = df_month_test[df_month_test['date'].isin(before_target[:-3])]
df_month_test_tmp

,time,IF00_close,IF01_close,spread,date,hour,minute,Expiration_Date,std,volatility,year,month,year_month,hour_minte
10122,2024-04-01 09:30:00,3555.6,3546.4,-9.2,2024-04-01,9,30,0,0.705752,-0.348491,2024,4,202404,09:30:00
10123,2024-04-01 09:31:00,3566.4,3559.8,-6.6,2024-04-01,9,31,0,0.705183,-0.349299,2024,4,202404,09:31:00
10124,2024-04-01 09:32:00,3573.8,3567.2,-6.6,2024-04-01,9,32,0,0.705459,-0.348907,2024,4,202404,09:32:00
10125,2024-04-01 09:33:00,3576.0,3568.2,-7.8,2024-04-01,9,33,0,0.712420,-0.339088,2024,4,202404,09:33:00
10126,2024-04-01 09:34:00,3576.0,3569.4,-6.6,2024-04-01,9,34,0,0.697141,-0.360768,2024,4,202404,09:34:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12286,2024-04-15 14:56:00,3544.2,3539.2,-5.0,2024-04-15,14,56,0,0.679095,-0.386994,2024,4,202404,14:56:00
12287,2024-04-15 14:57:00,3546.0,3539.0,-7.0,2024-04-15,14,57,0,0.684589,-0.378936,2024,4,202404,14:57:00
12288,2024-04-15 14:58:00,3545.6,3540.0,-5.6,2024-04-15,14,58,0,0.684235,-0.379455,2024,4,202404,14:58:00
12289,2024-04-15 14:59:00,3547.2,3541.4,-5.8,2024-04-15,14,59,0,0.684235,-0.379455,2024,4,202404,14:59:00


In [66]:
ks_2samp(df_month_target['volatility'].values, df_month_test_tmp.groupby('hour_minte')['volatility'].mean().values )
# dangtian  和t-3之前的均值

KstestResult(statistic=0.995850622406639, pvalue=2.1253371707422905e-141, statistic_location=-0.3127384371263485, statistic_sign=-1)

In [67]:
df_month_test_tmp2 = df_month_test[df_month_test['date'].isin(before_target[0:-3] )]
ks_2samp(df_month_test_tmp2.groupby('hour_minte')['volatility'].mean().values,\
    df_month_test[df_month_test['date']==before_target[-1]]['volatility'].values)

    # t-1  和t-3之前的均值

KstestResult(statistic=0.6016597510373444, pvalue=6.455411137873572e-41, statistic_location=-0.3129279655798802, statistic_sign=1)

In [68]:
df_month_test_tmp2 = df_month_test[df_month_test['date'].isin(before_target[0:-3] )]
ks_2samp(df_month_test_tmp2.groupby('hour_minte')['volatility'].mean().values,\
    df_month_test[df_month_test['date']==before_target[-2]]['volatility'].values)

    # t-2  和t-3之前的均值

KstestResult(statistic=0.8879668049792531, pvalue=5.338845026321695e-100, statistic_location=-0.3127384371263485, statistic_sign=1)

In [69]:
df_month_test_tmp2 = df_month_test[df_month_test['date'].isin(before_target[0:-3] )]
ks_2samp(df_month_test_tmp2.groupby('hour_minte')['volatility'].mean().values,\
    df_month_test[df_month_test['date']==after_target[0]]['volatility'].values)
    # t+ 1  与和t-3之前的均值

KstestResult(statistic=0.8257261410788381, pvalue=2.4300719653130242e-83, statistic_location=-0.3127384371263485, statistic_sign=1)

In [70]:
df_month_test_tmp2 = df_month_test[df_month_test['date'].isin(before_target[0:-3] )]
ks_2samp(df_month_test_tmp2.groupby('hour_minte')['volatility'].mean().values,\
    df_month_test[df_month_test['date']==after_target[1]]['volatility'].values)
    # t+2 和t-3之前的均值

KstestResult(statistic=0.5394190871369294, pvalue=1.7346775102876498e-32, statistic_location=-0.36348867167109367, statistic_sign=-1)

In [71]:
df_month_target

for i in range(1,3):
    for length in [15,30,60]:


        print(before_target[-i], f"前 {length} minutes",ks_2samp(df_month_test[df_month_test['date']==before_target[-i]]['volatility'].values[0:length],\
            df_month_target['volatility'].values[0:length]))



2024-04-18 前 15 minutes KstestResult(statistic=0.5333333333333333, pvalue=0.026248485664288602, statistic_location=-0.1268279303812454, statistic_sign=-1)
2024-04-18 前 30 minutes KstestResult(statistic=0.5666666666666667, pvalue=8.737803591291227e-05, statistic_location=-0.04714374342602712, statistic_sign=1)
2024-04-18 前 60 minutes KstestResult(statistic=0.7833333333333333, pvalue=1.8131121649108368e-18, statistic_location=-0.04208204336128161, statistic_sign=1)
2024-04-17 前 15 minutes KstestResult(statistic=0.6, pvalue=0.007655808318750842, statistic_location=-0.21268830282559842, statistic_sign=1)
2024-04-17 前 30 minutes KstestResult(statistic=0.36666666666666664, pvalue=0.034580084963334506, statistic_location=-0.11647259987651874, statistic_sign=1)
2024-04-17 前 60 minutes KstestResult(statistic=0.31666666666666665, pvalue=0.004627144187221645, statistic_location=0.13562924677765578, statistic_sign=-1)


In [72]:
for i in range(1,3):
    for length in [15,30,60]:


        print(before_target[-i], f"后 {length} minutes",ks_2samp(df_month_test[df_month_test['date']==before_target[-i]]['volatility'].values[-length:],\
            df_month_target['volatility'].values[-length:]))


2024-04-18 后 15 minutes KstestResult(statistic=1.0, pvalue=1.2893450075787698e-08, statistic_location=-0.4322402428504445, statistic_sign=1)
2024-04-18 后 30 minutes KstestResult(statistic=1.0, pvalue=1.6911233892144742e-17, statistic_location=-0.3853506811961316, statistic_sign=1)
2024-04-18 后 60 minutes KstestResult(statistic=1.0, pvalue=2.070073888186965e-35, statistic_location=-0.3853506811961316, statistic_sign=1)
2024-04-17 后 15 minutes KstestResult(statistic=1.0, pvalue=1.2893450075787698e-08, statistic_location=-0.13331673252572104, statistic_sign=1)
2024-04-17 后 30 minutes KstestResult(statistic=1.0, pvalue=1.6911233892144742e-17, statistic_location=-0.13331673252572104, statistic_sign=1)
2024-04-17 后 60 minutes KstestResult(statistic=1.0, pvalue=2.070073888186965e-35, statistic_location=-0.0440038808745317, statistic_sign=1)


In [73]:
df_month_target['minutes'] = df_month_target['hour']*60 + df_month_target['minute']
df_month_test['minutes'] = df_month_test['hour']*60 + df_month_test['minute']


time_ranges = [[13.0,14.0],[10.5,11.5]]

for i in range(1,3):
    for time_range in time_ranges:
        minutes_start = time_range[0]*60
        minutes_end = time_range[1]*60



        print(before_target[-i], f"{minutes_start}-{minutes_end}时间段内",  ks_2samp(df_month_test[(df_month_test['date']==before_target[-i])&
        (df_month_test['minutes']>=minutes_start)&(df_month_test['minutes']<minutes_end)]['volatility'].values,\
            df_month_target[(df_month_target['minutes']>=minutes_start)&(df_month_target['minutes']<minutes_end)]['volatility'].values))

2024-04-18 780.0-840.0时间段内 KstestResult(statistic=1.0, pvalue=8.211293089808295e-35, statistic_location=-0.17096637312234592, statistic_sign=1)
2024-04-18 630.0-690.0时间段内 KstestResult(statistic=1.0, pvalue=2.070073888186965e-35, statistic_location=-0.030158345922114718, statistic_sign=1)
2024-04-17 780.0-840.0时间段内 KstestResult(statistic=0.576271186440678, pvalue=2.143796495363778e-09, statistic_location=0.07869124711986938, statistic_sign=-1)
2024-04-17 630.0-690.0时间段内 KstestResult(statistic=1.0, pvalue=2.070073888186965e-35, statistic_location=0.11418043608476035, statistic_sign=-1)


/var/folders/71/8c25_pts4j92396n9h1t_88c0000gn/T/ipykernel_6613/3153899920.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_month_target['minutes'] = df_month_target['hour']*60 + df_month_target['minute']
